In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import os
import time
from tqdm import tqdm

In [3]:
file_path = os.listdir('cleaned_data')
# file_path

In [4]:
industry_name = [x[:-4] for x in file_path]

In [10]:
need_ = []
for f in range(len(file_path)):
    tmp = pd.read_csv('cleaned_data/'+file_path[f])
    need_.append(tmp)


In [11]:
whole = pd.concat(need_)

In [12]:
whole.to_csv('whole1.csv')

In [13]:
df = pd.read_csv('whole1.csv')

In [14]:
start_time = time.time()
total_dict = {}
# for j in tqdm(range(1)):
for j in tqdm(range(len(industry_name))):
    one_industry_index = {}
    # 选定一个行业
    one_industry = df[df['industry']==industry_name[j]]
    # 确定了行业之后确定这个行业所能覆盖的交易日
    days = one_industry['日期'].drop_duplicates()
    for i in range(len(days)):
        # 确定一个行业在一个交易日的信息
        one_industry_one_day = one_industry[one_industry['日期']==days.iloc[i]]
        # 计算每一天的weight
        one_industry_one_day['weight']=one_industry_one_day['总市值(元)']/sum(one_industry_one_day['总市值(元)'])
        # 依据weight和收盘价构造行业指数
        one_industry_one_day_index = sum(one_industry_one_day['收盘价(元)']*one_industry_one_day['weight'])
        one_industry_index[days.iloc[i]]=one_industry_one_day_index
    total_dict[industry_name[j]] = one_industry_index
end_time = time.time()
print('time claupse: ',end_time-start_time)


  0%|          | 0/79 [00:00<?, ?it/s]<ipython-input-14-3ec40dcf9abf>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_industry_one_day['weight']=one_industry_one_day['总市值(元)']/sum(one_industry_one_day['总市值(元)'])
100%|██████████| 79/79 [22:54<00:00, 17.40s/it]time claupse:  1374.4194731712341



In [15]:
# total_dict
all_index = pd.DataFrame(total_dict)

In [16]:
all_index.to_csv('all_year_index1.csv')

## corr 

In [17]:
all_data = pd.read_csv('all_year_index1.csv',index_col=0)

In [18]:
all_data['日期']=all_data.index

In [19]:
all_data['year'] = all_data.apply(lambda x:x['日期'][:-6],axis=1)

In [20]:
# all_data
years = all_data.year.drop_duplicates()
years

2010-01-04    2010
2011-01-04    2011
2012-01-04    2012
2013-01-04    2013
2014-01-02    2014
2015-01-05    2015
2016-01-04    2016
2017-01-03    2017
2018-01-02    2018
2019-01-02    2019
2020-01-02    2020
Name: year, dtype: object

In [21]:
for i in range(len(years)):
    year1 = all_data[all_data.year==years[i]][:-1]
    year1.set_index(year1['year'])
    year1 = year1.drop(['日期','year'],axis=1)
    dyear = year1.diff()/year1
    dyear = (dyear-dyear.mean())/dyear.std()
    corr = dyear.corr()
    corr.to_csv(years[i]+' corr.csv')